In [1]:
import pickle

from enbios.base.experiment import Experiment

In [2]:
# Open a pickled experiment object.
from pathlib import Path

# exp: Experiment = pickle.load(DataPath("test_data/exp.pkl").open("rb"))
exp: Experiment = pickle.load(Path("exp.pickle").open("rb"))

In [3]:
exp.structural_nodes_names

['electricity production, wind, 1-3MW turbine, onshore',
 'electricity production, wind, 1-3MW turbine, offshore',
 'electricity production, solar tower power plant, 20 MW',
 'electricity production, solar thermal parabolic trough, 50 MW']

In [4]:
alt_hierarchy = {
    "name": "root",
    "aggregator": "sum",
    "children": exp.structural_nodes_names,
}

In [9]:
rearranged_result_tree = exp.scenarios[0].result_to_dict(
    alternative_hierarchy=alt_hierarchy
)

rearranged_result_tree["results"]

{'GWP1000': {'unit': 'kg CO2-Eq', 'magnitude': 0.13466483029470988},
 'FETP': {'unit': 'kg 1,4-DCB-Eq', 'magnitude': 0.01735045631653643},
 'HTPnc': {'unit': 'kg 1,4-DCB-Eq', 'magnitude': 0.1368117616297974}}

In [10]:
exp.scenarios[0].result_to_dict(alternative_hierarchy=alt_hierarchy)

{'name': 'root',
 'results': {'GWP1000': {'unit': 'kg CO2-Eq',
   'magnitude': 0.13466483029470988},
  'FETP': {'unit': 'kg 1,4-DCB-Eq', 'magnitude': 0.01735045631653643},
  'HTPnc': {'unit': 'kg 1,4-DCB-Eq', 'magnitude': 0.1368117616297974}},
 'output': {'unit': 'kilowatt_hour', 'magnitude': 4.0},
 'children': [{'name': 'electricity production, wind, 1-3MW turbine, onshore',
   'results': {'GWP1000': {'unit': 'kg CO2-Eq',
     'magnitude': 0.015243611332791748},
    'FETP': {'unit': 'kg 1,4-DCB-Eq', 'magnitude': 0.0064256080745064845},
    'HTPnc': {'unit': 'kg 1,4-DCB-Eq', 'magnitude': 0.03545478768772618}},
   'output': {'unit': 'kilowatt_hour', 'magnitude': 1.0}},
  {'name': 'electricity production, wind, 1-3MW turbine, offshore',
   'results': {'GWP1000': {'unit': 'kg CO2-Eq',
     'magnitude': 0.017020448257678778},
    'FETP': {'unit': 'kg 1,4-DCB-Eq', 'magnitude': 0.004589452362081367},
    'HTPnc': {'unit': 'kg 1,4-DCB-Eq', 'magnitude': 0.046378418526606936}},
   'output': {'u

In [11]:
import pandas as pd

exp.scenarios[0].results_to_csv("s1.csv", level_names=["root", "tech", "activity"])
pd.read_csv("s1.csv").fillna("")

,root,tech,activity,GWP1000_magnitude (kg CO2-Eq),"FETP_magnitude (kg 1,4-DCB-Eq)","HTPnc_magnitude (kg 1,4-DCB-Eq)",output_unit,output_magnitude
0,root,,,0.134665,0.017350,0.136812,kilowatt_hour,4.0
1,,wind,,0.032264,0.011015,0.081833,kilowatt_hour,2.0
2,,,"electricity production, wind, 1-3MW turbine, o...",0.015244,0.006426,0.035455,kilowatt_hour,1.0
3,,,"electricity production, wind, 1-3MW turbine, o...",0.017020,0.004589,0.046378,kilowatt_hour,1.0
4,,solar,,0.102401,0.006335,0.054979,kilowatt_hour,2.0
5,,,"electricity production, solar tower power plan...",0.048205,0.003242,0.027197,kilowatt_hour,1.0
6,,,"electricity production, solar thermal paraboli...",0.054196,0.003093,0.027781,kilowatt_hour,1.0


In [12]:
import pandas as pd

exp.scenarios[0].results_to_csv(
    "s1.csv", alternative_hierarchy=alt_hierarchy, level_names=["root", "activity"]
)
pd.read_csv("s1.csv").fillna("")

,root,activity,GWP1000_magnitude (kg CO2-Eq),"FETP_magnitude (kg 1,4-DCB-Eq)","HTPnc_magnitude (kg 1,4-DCB-Eq)",output_unit,output_magnitude
0,root,,0.134665,0.017350,0.136812,kilowatt_hour,4.0
1,,"electricity production, wind, 1-3MW turbine, o...",0.015244,0.006426,0.035455,kilowatt_hour,1.0
2,,"electricity production, wind, 1-3MW turbine, o...",0.017020,0.004589,0.046378,kilowatt_hour,1.0
3,,"electricity production, solar tower power plan...",0.048205,0.003242,0.027197,kilowatt_hour,1.0
4,,"electricity production, solar thermal paraboli...",0.054196,0.003093,0.027781,kilowatt_hour,1.0
